In [1]:
import pandas as pd
matches = pd.read_csv("matches.csv", index_col = 0)
matches.shape

(1389, 27)

In [2]:
matches['team'].value_counts()

Southampton                 72
Brighton and Hove Albion    72
Manchester United           72
West Ham United             72
Newcastle United            72
Burnley                     71
Leeds United                71
Crystal Palace              71
Manchester City             71
Wolverhampton Wanderers     71
Tottenham Hotspur           71
Arsenal                     71
Leicester City              70
Chelsea                     70
Aston Villa                 70
Everton                     70
Liverpool                   38
Fulham                      38
West Bromwich Albion        38
Sheffield United            38
Brentford                   34
Watford                     33
Norwich City                33
Name: team, dtype: int64

In [3]:
matches["round"].value_counts()

Matchweek 1     39
Matchweek 16    39
Matchweek 34    39
Matchweek 32    39
Matchweek 31    39
Matchweek 29    39
Matchweek 28    39
Matchweek 26    39
Matchweek 25    39
Matchweek 24    39
Matchweek 23    39
Matchweek 2     39
Matchweek 19    39
Matchweek 17    39
Matchweek 20    39
Matchweek 15    39
Matchweek 5     39
Matchweek 3     39
Matchweek 13    39
Matchweek 12    39
Matchweek 4     39
Matchweek 11    39
Matchweek 10    39
Matchweek 9     39
Matchweek 8     39
Matchweek 14    39
Matchweek 7     39
Matchweek 6     39
Matchweek 30    37
Matchweek 27    37
Matchweek 22    37
Matchweek 21    37
Matchweek 18    37
Matchweek 33    32
Matchweek 35    20
Matchweek 36    20
Matchweek 37    20
Matchweek 38    20
Name: round, dtype: int64

In [4]:
matches["date"] = pd.to_datetime(matches["date"])

In [5]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+", "", regex = True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek
matches["target"] = (matches["result"] == "W").astype("int")

In [6]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=2)
train = matches[matches["date"] < "2022-01-01"]
test = matches[matches["date"] > "2022-01-01"]
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [7]:
rf.fit(train[predictors],train["target"])
preds = rf.predict(test[predictors])

In [8]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(test["target"], preds)
acc

0.6159420289855072

In [9]:
combined = pd.DataFrame(dict(actual=test["target"], prediction = preds))
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,143,29
1,77,27


In [10]:
from sklearn.metrics import precision_score
precision_score(test["target"],preds)


0.48214285714285715

In [11]:
grouped_matches = matches.groupby("team")
group = grouped_matches.get_group("Liverpool")

In [12]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group
cols = ["gf","ga","sh","sot","dist","fk","pk","pkatt"]
new_cols = [f"{c}_rolling" for c in cols]
rolling_averages(group,cols,new_cols)



,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
6,2020-10-04,19:15,Premier League,Matchweek 4,Sun,Away,L,2.0,7.0,Aston Villa,...,6,0,3.000000,1.333333,19.333333,6.000000,17.166667,0.333333,0.666667,0.666667
7,2020-10-17,12:30,Premier League,Matchweek 5,Sat,Away,D,2.0,2.0,Everton,...,5,0,2.333333,2.666667,17.333333,7.333333,16.766667,0.666667,0.000000,0.000000
9,2020-10-24,20:00,Premier League,Matchweek 6,Sat,Home,W,2.0,1.0,Sheffield Utd,...,5,1,2.333333,3.333333,19.000000,8.333333,15.866667,0.666667,0.000000,0.000000
11,2020-10-31,17:30,Premier League,Matchweek 7,Sat,Home,W,2.0,1.0,West Ham,...,5,1,2.000000,3.333333,17.666667,7.000000,16.333333,1.000000,0.000000,0.000000
13,2020-11-08,16:30,Premier League,Matchweek 8,Sun,Away,D,1.0,1.0,Manchester City,...,6,0,2.000000,1.333333,15.666667,5.000000,17.266667,1.000000,0.333333,0.333333
14,2020-11-22,19:15,Premier League,Matchweek 9,Sun,Home,W,3.0,0.0,Leicester City,...,6,1,1.666667,1.000000,11.333333,3.000000,19.433333,0.666667,0.666667,0.666667
16,2020-11-28,12:30,Premier League,Matchweek 10,Sat,Away,D,1.0,1.0,Brighton,...,5,0,2.000000,0.666667,13.666667,5.333333,17.333333,0.333333,0.666667,0.666667
18,2020-12-06,19:15,Premier League,Matchweek 11,Sun,Home,W,4.0,0.0,Wolves,...,6,1,1.666667,0.666667,13.000000,5.333333,18.100000,0.000000,0.333333,0.333333
20,2020-12-13,16:30,Premier League,Matchweek 12,Sun,Away,D,1.0,1.0,Fulham,...,6,0,2.666667,0.333333,13.666667,6.666667,16.466667,0.333333,0.000000,0.000000
21,2020-12-16,20:00,Premier League,Matchweek 13,Wed,Home,W,2.0,1.0,Tottenham,...,2,1,2.000000,0.666667,9.333333,4.333333,19.166667,0.666667,0.333333,0.333333


In [13]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x,cols,new_cols))
matches_rolling = matches_rolling.droplevel("team")
matches_rolling.index = range(matches_rolling.shape[0])

In [14]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error

In [15]:
combined, error = make_predictions(matches_rolling, predictors + new_cols)
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)
combined

,actual,predicted,date,team,opponent,result
55,0,0,2022-01-23,Arsenal,Burnley,D
56,1,1,2022-02-10,Arsenal,Wolves,W
57,1,0,2022-02-19,Arsenal,Brentford,W
58,1,1,2022-02-24,Arsenal,Wolves,W
59,1,1,2022-03-06,Arsenal,Watford,W
...,...,...,...,...,...,...
1312,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W
1313,0,0,2022-03-18,Wolverhampton Wanderers,Leeds United,L
1314,1,0,2022-04-02,Wolverhampton Wanderers,Aston Villa,W
1315,0,1,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L


In [16]:

#from pathlib import Path  
#filepath = Path('C:\Coding\IA Website build\Test\predictions.csv')  
#filepath.parent.mkdir(parents=True, exist_ok=True)  
#combined.to_csv(filepath)  
